In [1]:
import xarray as xr
from dask.distributed import Client

In [10]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]
area = ((1-tmask) * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'depth', 'y':'gridY', 'x':'gridX'})

In [12]:
chunk_sizes = {'time': 1, 'depth': 40, 'gridY': 898, 'gridX': 398}

In [16]:
year = 2019
fname = 'SalishSeaCast_river_trace_day_avg_allR_00_40_res1M_{}0101_{}1231_e3t_3d_vol'.format(year, year)
volume = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_vol/{}.nc'.format(fname),chunks=chunk_sizes)

In [27]:
volume

<xarray.Dataset>
Dimensions:          (depth: 40, gridY: 898, gridX: 398, time: 12)
Coordinates:
  * depth            (depth) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
  * gridY            (gridY) int64 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX            (gridX) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396 397
  * time             (time) datetime64[ns] 2019-01-15T12:00:00 ... 2019-12-15...
Data variables: (12/13)
    nooksack_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    skagit_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    snohomish_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    nisqually_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    elwha_river      (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    cowichan_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    ...               ...
    puntledge_river  (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    salmon_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    homathko_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    squamish_river   (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    fraser_river     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    other_rivers     (time, depth, gridY, gridX) float64 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>

In [ ]:
client=Client(processes=True, n_workers=4, local_directory='/tmp/cdonaldson')  # 4 is a good choice for n_workers even on salish, if I was the only person logged on or asked for space, could go as high as 16 but... be conscious of this
# threads_per_worker = 1 if needed, netcdf is not threadsafe now so be realiable by specifying 1 (at expense of just some efficiency)

In [19]:
meters = volume / area

In [ ]:
for data_var in list(meters.data_vars):
    meters[data_var].attrs['units'] = 'm'

In [ ]:
fname_save = fname[:-10] + 'meters_3d'
fname_save

In [ ]:
meters.to_netcdf('/ocean/cdonaldson/research_runs/hindcast_auto/transformations/3d_hindcast_meters/{}.nc'.format(fname_save), engine="netcdf4")

In [ ]:
client.close()